# Generate Questions for Benchmark

In [1]:
import subreddit_question_generation as sub_gen

In [2]:
sub_gen.generate_for_benchmark()

0it [00:12, ?it/s]


KeyboardInterrupt: 

# End-to-end

In [1]:
from openai import OpenAI
import pandas as pd
from secret_retriever import retrieve_secret

# initializing openai configuration
BASE_URL = retrieve_secret('cs4973_base_url')
API_KEY=api_key = retrieve_secret('cs4973_api_key')
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

In [2]:
from final import get_top_post, get_comments_of_post, get_only_opinionated_comments

/scratch/bchk/kbengani/reddit_sentiment_analysis/fact_opinion_classification/classifier.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load

In [3]:
query = "At NEU, how do people feel about Snell?"

post = get_top_post(query)
print(post.title)

Searching At NEU, people feel Snell?
Why do people make so much noise at Snell Library?


In [4]:
comments = get_comments_of_post(post)
comments = [comment.body for comment in comments]

In [5]:
opinionated_comments = get_only_opinionated_comments(comments)

In [6]:
from langchain.text_splitter import CharacterTextSplitter

def generate_chunks(context_documents):
    context = '#####'.join(context_documents)
    text_splitter = CharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 0,
        length_function=len,
        separator = '\n\n'
    )
    return text_splitter.split_text(context)

In [7]:
chunked_comments = generate_chunks(opinionated_comments[::-1])

Created a chunk of size 589, which is longer than the specified 500
Created a chunk of size 640, which is longer than the specified 500


In [8]:
chunked_post = generate_chunks([post.title, post.selftext])

In [ ]:

SYSTEM_PROMPT = """
Given the post and its respective comments as context, return whether 
the overall opinion on the given question is "positive", "negative", or "neutral".
"""

messages = [{'role': 'system', 'content': SYSTEM_PROMPT}]
messages.extend([{'role': 'user', 'content': 'COMMENTS:'}])
messages.extend([{'role': 'user', 'content': chunk} for chunk in chunked_comments])
messages.extend([{'role': 'user', 'content': 'POST:'}])
messages.extend([{'role': 'user', 'content': chunk} for chunk in chunked_post])
messages.extend([{'role': 'user', 'content': f'QUESTION: {query}'}])
print("HERE")
resp = client.chat.completions.create(
    messages = messages,
    model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0.7)

model_response = resp.choices[0].message.content[0]

HERE


In [ ]:
model_response